In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os, glob

In [3]:
path = os.path.join(os.path.dirname(os.getcwd()), '../Data/Extracted_data/')
os.chdir(path)

### MBN

In [21]:
MBN = pd.read_csv('MBN.csv', index_col='Sample')
MBN

,location,measure,peak,peak_ref,fwhm,fwhm_ref,diff_two_peaks,diff_two_peaks_ref,amp,amp_ref,pos,pos_ref,energy,energy_ref
Sample,,,,,,,,,,,,,,
41C,1,1,0.267420,0.164482,0.033634,0.019858,0.035758,0.025778,0.015838,0.013743,7686.0,7624,408.764909,273.907729
41C,1,2,0.220200,0.164482,0.029495,0.019858,0.052288,0.025778,0.017809,0.013743,7642.0,7624,407.653684,273.907729
41C,1,3,0.222581,0.164482,0.028561,0.019858,0.048347,0.025778,0.014434,0.013743,7638.0,7624,395.134422,273.907729
41C,2,1,0.269990,0.164482,0.034059,0.019858,0.050624,0.025778,0.018987,0.013743,7677.0,7624,390.800935,273.907729
41C,2,2,0.246733,0.164482,0.033052,0.019858,0.060581,0.025778,0.020128,0.013743,7642.0,7624,415.882398,273.907729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6C,5,2,0.266634,0.177791,0.030577,0.016124,0.055583,0.025268,0.014134,0.010430,7588.0,7684,356.821249,249.070282
6C,5,3,0.219244,0.177791,0.022055,0.016124,0.037694,0.025268,0.012990,0.010430,7604.0,7684,326.323321,249.070282
6C,6,1,0.194222,0.177791,0.024528,0.016124,0.042488,0.025268,0.013571,0.010430,7574.0,7684,337.986332,249.070282


### NLE

In [73]:
NLE = pd.read_csv('NLE.csv')
NLE['NLE_avg'] = NLE[['S10', 'S11', 'S12']].mean(axis=1)
NLE = pd.pivot_table(NLE, index='Sample' ,columns='freq', values='NLE_avg')
NLE = NLE.add_prefix('NLE_avg')
NLE 

freq,NLE_avg17,NLE_avg51,NLE_avg85,NLE_avg119
Sample,,,,
10C,0.455524,0.418210,0.512854,0.228678
13C,0.462425,0.425272,0.509204,0.216304
14C,0.455159,0.411063,0.491542,0.215602
16C,0.448236,0.403479,0.472269,0.197359
19C,0.470000,0.460072,0.580688,0.249148
20C,0.454961,0.407136,0.490481,0.215616
21C,0.455361,0.423233,0.509838,0.224605
23C,0.456454,0.419344,0.508837,0.226340
24C,0.471773,0.398403,0.486216,0.206717


### Electric

In [84]:
electric = pd.read_csv('Electric_RP.csv', index_col='ID')
electric.drop('Unnamed: 0', axis=1, inplace=True)
electric

,ACEE_30_Avg_38,ACEE_500_Avg_38,Avg_EFC,Avg_RP,ACPD_Avg_38,ACEE_30_Avg_58,ACEE_500_Avg_58,ACPD_Avg_58
ID,,,,,,,,
41C,-0.249983,-2.664717,97.42400,190.979294,3.726051,-0.137640,-2.866030,4.804749
43C,-0.396353,-2.390293,95.83600,184.930215,3.064434,-0.462040,-2.254125,3.869428
44C,-0.499607,-2.747297,96.43600,187.204261,2.720381,-0.596385,-2.788500,3.370729
45C,-0.582543,-3.051327,96.91000,189.010633,3.064732,-0.645710,-3.202220,4.186639
7C,-1.079600,-1.504297,94.28700,179.123964,2.957302,-1.135960,-1.718115,3.438451
13C,-0.381817,-1.541673,91.83700,170.130491,3.078895,-0.660875,-2.167395,3.725514
8C,-0.878553,-1.530483,91.76400,169.866095,2.787400,-0.951840,-1.502545,3.529319
35C,-0.813900,-1.298970,92.16500,171.321012,2.647839,-0.867430,-1.350065,3.050650
24C,-0.627700,-1.985753,93.02300,174.454983,3.041253,-0.816755,-1.836935,4.259982


### NLO

In [85]:
NLO = pd.read_csv('NLO_result.csv', index_col='Sample')
NLO.drop('Stress', axis=1, inplace=True)

### Fatigue

In [86]:
fatigue = pd.read_excel('FatigueSampleInventory.xlsx', 
                        index_col=' Specimen ID',
                        usecols=[' Specimen ID', 'Cycles at NDE Test', 'Cycles at Failure', 'Current Unfailed Cycles']
                       )
fatigue['f'] = fatigue['Cycles at NDE Test']/fatigue['Cycles at Failure']
fatigue

,Cycles at NDE Test,Cycles at Failure,Current Unfailed Cycles,f
Specimen ID,,,,
41C,0,2154137.0,NaN,0.000000
43C,0,NaN,NaN,NaN
44C,0,1318034.0,NaN,0.000000
45C,0,NaN,3245267.0,NaN
7C,150000,NaN,NaN,NaN
13C,150000,NaN,NaN,NaN
8C,150000,NaN,NaN,NaN
35C,150000,NaN,NaN,NaN
24C,300000,NaN,NaN,NaN


Merge all data

In [87]:
merged = fatigue.join([NLE, NLO, electric], how='left')

In [88]:
merged.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Merged_data/MERGE_on_sample.csv'))

In [89]:
merged

,Cycles at NDE Test,Cycles at Failure,Current Unfailed Cycles,f,NLE_avg17,NLE_avg51,NLE_avg85,NLE_avg119,Average,St Dev,Confidence,ACEE_30_Avg_38,ACEE_500_Avg_38,Avg_EFC,Avg_RP,ACPD_Avg_38,ACEE_30_Avg_58,ACEE_500_Avg_58,ACPD_Avg_58
10C,300000,NaN,NaN,NaN,0.455524,0.418210,0.512854,0.228678,4.240700,24.965134,4.604018,-0.804607,-1.779930,92.24800,171.622934,1.993660,-0.797640,-1.795000,2.579638
13C,150000,NaN,NaN,NaN,0.462425,0.425272,0.509204,0.216304,10.196998,25.869459,3.635891,-0.381817,-1.541673,91.83700,170.130491,3.078895,-0.660875,-2.167395,3.725514
13C,150000,NaN,NaN,NaN,0.462425,0.425272,0.509204,0.216304,9.082132,26.491178,2.895681,-0.381817,-1.541673,91.83700,170.130491,3.078895,-0.660875,-2.167395,3.725514
13C,150000,NaN,NaN,NaN,0.462425,0.425272,0.509204,0.216304,9.784843,28.676441,3.324709,-0.381817,-1.541673,91.83700,170.130491,3.078895,-0.660875,-2.167395,3.725514
14C,2000000,NaN,NaN,NaN,0.455159,0.411063,0.491542,0.215602,4.588220,24.327394,4.597903,-0.847057,-0.847057,88.81200,159.347604,2.515683,-0.828250,-0.828250,3.291392
16C,2000000,NaN,NaN,NaN,0.448236,0.403479,0.472269,0.197359,3.755165,21.280844,3.846587,-1.048487,-1.769470,91.67900,169.558497,1.623430,-1.108970,-1.808550,2.446152
19C,450000,914392.0,NaN,0.492130,0.470000,0.460072,0.580688,0.249148,3.798736,22.043955,4.164618,-0.927853,-1.458537,92.33100,171.925123,2.473902,-1.065155,-1.570740,3.226957
20C,2000000,NaN,NaN,NaN,0.454961,0.407136,0.490481,0.215616,3.791726,22.269871,4.147796,-0.795270,-0.795270,91.80700,170.021810,2.399208,-1.223960,-1.223960,2.646820
21C,600000,800007.0,NaN,0.749993,0.455361,0.423233,0.509838,0.224605,4.626314,26.268556,4.982828,-0.865243,-1.609157,92.90000,174.003954,2.305978,-0.884900,-1.578795,2.764862
23C,600000,NaN,2855225.0,NaN,0.456454,0.419344,0.508837,0.226340,5.621833,26.908791,4.986986,-0.574873,-0.985490,92.73600,173.403495,2.536283,-0.831110,-1.194540,3.061743
